In [9]:
# Lab 3: Build fixpoint calculator using LSTM Networks
# Tara Zamani 

from random import seed
from random import randint
from random import uniform
import numpy as np
from math import ceil
from math import log10
from math import sqrt
from numpy import argmax
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import TimeDistributed
from tensorflow.keras.layers import RepeatVector

In [10]:
# Task 1
# Dataset preparation (format)
# 0) Let's use [9:5]_2.[4:0]_2 as the UN-SIGNED fix-point representation 
#    for example, 01000.10000 = 2^3.5 = 8.5 (UPDATED: NOW totally 10 bits)
# Requirement
# 1) Need to have seperate functions, including (check out bracket-2 in lab3-demo)
#    1-a) data generation
#    1-b) data conversion and inversion (inversion for inference/testing use)

In [11]:
#Helper Functions

# convert all list elements from floating point to 10-bit fixed point
def num_to_fixed(input_list):
    ans = []
    for i in input_list:
        #tansforming floating point number to an integer by shifting decimal 5 digits to the right
        temp = int(i * 32)
        fixed_num = ""
        for digit in range(10):
            if (temp%2 == 0):
                fixed_num = '0' + fixed_num
            else:
                fixed_num = '1' + fixed_num
            temp = int(temp / 2) 
        ans.append(fixed_num)
    return ans

# convert from 10-bit fixed point string to floating point
def fixed_to_num(string):
    ans = ""
    if '+' in string: # --> inverting x
        a,b = string.split('+')
        num1, num2 = 0, 0
        for i in range(10):
            num1 += float(int(a[i])*2**(4-i))
            num2 += float(int(b[i])*2**(4-i))
        ans = str(num1) + "+" + str(num2)
    else: # --> inverting y
        num = 0
        for i in range(10):
            num += float(int(string[i])*2**(4-i))
        ans = str(num)    
    return ans

# generate lists of random 10-bit fixed point numbers and their sum
def random_sum_pairs(n_examples, n_numbers, largest):
    X, y = list(), list()
    for i in range(n_examples):
        # generate floating point numbers by separetely creating the 5-bit decimal and mantissa
        in_pattern = [(float(randint(1, largest)) + randint(1, largest)/32.0) for _ in range(n_numbers)]
        out_pattern = sum(in_pattern)
        X.append(in_pattern)
        y.append(out_pattern)
    return X, y

# convert data to strings
def to_string(X, y, n_numbers, largest):
    max_length = 21
    Xstr = list()
    for pattern in X:
        strp = '+'.join([str(n) for n in pattern])
        strp = ''.join([' ' for _ in range(int(max_length-len(strp)))]) + strp
        Xstr.append(strp)
    max_length = 10
    ystr = list()
    for pattern in y:
        strp = str(pattern)
        strp = ''.join([' ' for _ in range(int(max_length-len(strp)))]) + strp
        ystr.append(strp)
    return Xstr, ystr

# integer encode strings
def integer_encode(X, y, alphabet):
    char_to_int = dict((c, i) for i, c in enumerate(alphabet))
    Xenc = list()
    for pattern in X:
        integer_encoded = [char_to_int[char] for char in pattern]
        Xenc.append(integer_encoded)
    yenc = list()
    for pattern in y:
        integer_encoded = [char_to_int[char] for char in pattern]
        yenc.append(integer_encoded)
    return Xenc, yenc
 
# one hot encode
def one_hot_encode(X, y, max_int):
    Xenc = list()
    for seq in X:
        pattern = list()
        for index in seq:
            vector = [0 for _ in range(max_int)]
            vector[index] = 1
            pattern.append(vector)
        Xenc.append(pattern)
    yenc = list()
    for seq in y:
        pattern = list()
        for index in seq:
            vector = [0 for _ in range(max_int)]
            vector[index] = 1
            pattern.append(vector)
        yenc.append(pattern)
    return Xenc, yenc

In [12]:
# generate an encoded dataset
def generate_data(n_samples, n_numbers, largest, alphabet):
    # generate pairs
    X, y = random_sum_pairs(n_samples, n_numbers, largest)
    # convert floating point list to fixed point list
    X = [num_to_fixed(a) for a in X]
    y = num_to_fixed(y)
    # convert to strings
    X, y = to_string(X, y, n_numbers, largest)
    # integer encode
    X, y = integer_encode(X, y, alphabet)
    # one hot encode
    X, y = one_hot_encode(X, y, len(alphabet))
    # return as numpy arrays
    X, y = np.array(X), np.array(y)
    return X, y

# invert encoding
def invert(seq, alphabet):
    int_to_char = dict((i, c) for i, c in enumerate(alphabet))
    strings = list()
    for pattern in seq:
        string = int_to_char[argmax(pattern)]
        strings.append(string)
    fixedpnt_str = ''.join(strings)
    return fixed_to_num(fixedpnt_str)

In [13]:
# define dataset
seed(1)
n_samples = 1000
n_numbers = 2
largest = 16
#alphabet = ['0', '1', '2', '3', '4', '5', '6', '7', '8', '9', '+', ' ', '.']
alphabet = ['0', '1', '+', ' ']
n_chars = len(alphabet)
n_in_seq_length = 21 #13
n_out_seq_length = 10 #6

In [14]:
X_train, y_train = generate_data(n_samples, n_numbers, largest, alphabet)
# check your dataset sample
print("XTrain\n",X_train[0],"\nYTrain\n",y_train[0])
print("\nInverted XTrain",invert(X_train[0],alphabet),"\nInverted YTrain",invert(y_train[0],alphabet))
# check your dataset sample
print("\nEX2:\nXTrain\n",X_train[1],"\nYTrain\n",y_train[1])
print("\nInverted XTrain",invert(X_train[1],alphabet),"\nInverted YTrain",invert(y_train[1],alphabet))

XTrain
 [[1 0 0 0]
 [1 0 0 0]
 [0 1 0 0]
 [1 0 0 0]
 [0 1 0 0]
 [1 0 0 0]
 [1 0 0 0]
 [1 0 0 0]
 [0 1 0 0]
 [0 1 0 0]
 [0 0 1 0]
 [1 0 0 0]
 [0 1 0 0]
 [1 0 0 0]
 [1 0 0 0]
 [0 1 0 0]
 [1 0 0 0]
 [1 0 0 0]
 [0 1 0 0]
 [1 0 0 0]
 [1 0 0 0]] 
YTrain
 [[1 0 0 0]
 [0 1 0 0]
 [0 1 0 0]
 [0 1 0 0]
 [1 0 0 0]
 [1 0 0 0]
 [1 0 0 0]
 [0 1 0 0]
 [0 1 0 0]
 [0 1 0 0]]

Inverted XTrain 5.09375+9.125 
Inverted YTrain 14.21875

EX2:
XTrain
 [[0 1 0 0]
 [1 0 0 0]
 [1 0 0 0]
 [1 0 0 0]
 [1 0 0 0]
 [1 0 0 0]
 [0 1 0 0]
 [0 1 0 0]
 [0 1 0 0]
 [0 1 0 0]
 [0 0 1 0]
 [0 1 0 0]
 [1 0 0 0]
 [1 0 0 0]
 [1 0 0 0]
 [1 0 0 0]
 [1 0 0 0]
 [0 1 0 0]
 [0 1 0 0]
 [1 0 0 0]
 [0 1 0 0]] 
YTrain
 [[1 0 0 0]
 [1 0 0 0]
 [1 0 0 0]
 [1 0 0 0]
 [1 0 0 0]
 [0 1 0 0]
 [0 1 0 0]
 [0 1 0 0]
 [1 0 0 0]
 [1 0 0 0]]

Inverted XTrain 16.46875+16.40625 
Inverted YTrain 0.875


In [15]:
# Task 2
# Build and Train a TWO-LAYER LSTM Network (two stacked LSTM).
# 1) Feel free to use any optimizers, normlization techniques, decaying, etc., techniques
# 2) Your training dataset should be LESS THAN 50% of the total data points
# 3) In the testing phase, you should implement (check out bracekt-8 in lab3-demo)
#  3-a) Inference with accuracy as metrics
#. 3-b) Inference with MSE as metrics
#. 3-c) Showing at least 50 cases (individual inference)

In [16]:
# define LSTM configuration
n_batch = 20
n_epoch = 400
# create LSTM
model = Sequential()
model.add(LSTM(64, input_shape=(n_in_seq_length, n_chars)))
model.add(RepeatVector(int(n_out_seq_length)))
model.add(LSTM(32, return_sequences=True))
model.add(TimeDistributed(Dense(n_chars, activation='softmax')))

optimizer = tf.keras.optimizers.Adam(lr=0.001)
model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])
print(model.summary())

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_2 (LSTM)                (None, 64)                17664     
_________________________________________________________________
repeat_vector_1 (RepeatVecto (None, 10, 64)            0         
_________________________________________________________________
lstm_3 (LSTM)                (None, 10, 32)            12416     
_________________________________________________________________
time_distributed_1 (TimeDist (None, 10, 4)             132       
Total params: 30,212
Trainable params: 30,212
Non-trainable params: 0
_________________________________________________________________
None


In [17]:
# train LSTM
model.fit(X_train, y_train, epochs=n_epoch, batch_size=n_batch, verbose=1)

Train on 1000 samples
Epoch 1/400
1000/1000 [==============================] - 4s 4ms/sample - loss: 0.8691 - accuracy: 0.5056
Epoch 2/400
1000/1000 [==============================] - 1s 828us/sample - loss: 0.6971 - accuracy: 0.5114
Epoch 3/400
1000/1000 [==============================] - 1s 850us/sample - loss: 0.6946 - accuracy: 0.5132
Epoch 4/400
1000/1000 [==============================] - 1s 825us/sample - loss: 0.6938 - accuracy: 0.5102
Epoch 5/400
1000/1000 [==============================] - 1s 1ms/sample - loss: 0.6935 - accuracy: 0.5114
Epoch 6/400
1000/1000 [==============================] - 1s 1ms/sample - loss: 0.6926 - accuracy: 0.5162
Epoch 7/400
1000/1000 [==============================] - 1s 1ms/sample - loss: 0.6930 - accuracy: 0.5110
Epoch 8/400
1000/1000 [==============================] - 1s 1ms/sample - loss: 0.6926 - accuracy: 0.5112
Epoch 9/400
1000/1000 [==============================] - 1s 1ms/sample - loss: 0.6924 - accuracy: 0.5099
Epoch 10/400
1000/1000 [===

Epoch 77/400
1000/1000 [==============================] - 1s 782us/sample - loss: 0.6147 - accuracy: 0.5906
Epoch 78/400
1000/1000 [==============================] - 1s 793us/sample - loss: 0.6062 - accuracy: 0.5935
Epoch 79/400
1000/1000 [==============================] - 1s 832us/sample - loss: 0.6087 - accuracy: 0.5921
Epoch 80/400
1000/1000 [==============================] - 1s 1ms/sample - loss: 0.6129 - accuracy: 0.5877
Epoch 81/400
1000/1000 [==============================] - 1s 797us/sample - loss: 0.6101 - accuracy: 0.5905
Epoch 82/400
1000/1000 [==============================] - 1s 826us/sample - loss: 0.5974 - accuracy: 0.6027
Epoch 83/400
1000/1000 [==============================] - 1s 809us/sample - loss: 0.5945 - accuracy: 0.6001
Epoch 84/400
1000/1000 [==============================] - 1s 979us/sample - loss: 0.5968 - accuracy: 0.6003
Epoch 85/400
1000/1000 [==============================] - 1s 1ms/sample - loss: 0.5908 - accuracy: 0.6046
Epoch 86/400
1000/1000 [========

1000/1000 [==============================] - 1s 780us/sample - loss: 0.4758 - accuracy: 0.7022
Epoch 153/400
1000/1000 [==============================] - 1s 778us/sample - loss: 0.4874 - accuracy: 0.6966
Epoch 154/400
1000/1000 [==============================] - 1s 791us/sample - loss: 0.4732 - accuracy: 0.7068
Epoch 155/400
1000/1000 [==============================] - 1s 741us/sample - loss: 0.4960 - accuracy: 0.6941
Epoch 156/400
1000/1000 [==============================] - 1s 759us/sample - loss: 0.5273 - accuracy: 0.6705
Epoch 157/400
1000/1000 [==============================] - 1s 787us/sample - loss: 0.4712 - accuracy: 0.7085
Epoch 158/400
1000/1000 [==============================] - 1s 807us/sample - loss: 0.4659 - accuracy: 0.7129
Epoch 159/400
1000/1000 [==============================] - 1s 789us/sample - loss: 0.4645 - accuracy: 0.7116
Epoch 160/400
1000/1000 [==============================] - 1s 785us/sample - loss: 0.4600 - accuracy: 0.7144
Epoch 161/400
1000/1000 [========

1000/1000 [==============================] - 1s 912us/sample - loss: 0.4028 - accuracy: 0.7511
Epoch 228/400
1000/1000 [==============================] - 1s 834us/sample - loss: 0.3999 - accuracy: 0.7550
Epoch 229/400
1000/1000 [==============================] - 1s 775us/sample - loss: 0.4045 - accuracy: 0.7513
Epoch 230/400
1000/1000 [==============================] - 1s 752us/sample - loss: 0.4672 - accuracy: 0.7171
Epoch 231/400
1000/1000 [==============================] - 1s 837us/sample - loss: 0.5218 - accuracy: 0.6941
Epoch 232/400
1000/1000 [==============================] - 1s 1ms/sample - loss: 0.4392 - accuracy: 0.7304
Epoch 233/400
1000/1000 [==============================] - 1s 766us/sample - loss: 0.4072 - accuracy: 0.7531
Epoch 234/400
1000/1000 [==============================] - 1s 821us/sample - loss: 0.4069 - accuracy: 0.7503
Epoch 235/400
1000/1000 [==============================] - 1s 943us/sample - loss: 0.4033 - accuracy: 0.7507
Epoch 236/400
1000/1000 [==========

1000/1000 [==============================] - 1s 1ms/sample - loss: 0.3515 - accuracy: 0.7923
Epoch 303/400
1000/1000 [==============================] - 1s 1ms/sample - loss: 0.3459 - accuracy: 0.7981
Epoch 304/400
1000/1000 [==============================] - 1s 1ms/sample - loss: 0.4170 - accuracy: 0.7684
Epoch 305/400
1000/1000 [==============================] - 1s 881us/sample - loss: 0.4199 - accuracy: 0.7570
Epoch 306/400
1000/1000 [==============================] - 1s 810us/sample - loss: 0.3737 - accuracy: 0.7822
Epoch 307/400
1000/1000 [==============================] - 1s 806us/sample - loss: 0.3520 - accuracy: 0.7926
Epoch 308/400
1000/1000 [==============================] - 1s 743us/sample - loss: 0.3443 - accuracy: 0.7974
Epoch 309/400
1000/1000 [==============================] - 1s 755us/sample - loss: 0.3418 - accuracy: 0.8015
Epoch 310/400
1000/1000 [==============================] - 1s 747us/sample - loss: 0.3421 - accuracy: 0.8019
Epoch 311/400
1000/1000 [==============

1000/1000 [==============================] - 1s 1ms/sample - loss: 0.3110 - accuracy: 0.8312
Epoch 378/400
1000/1000 [==============================] - 1s 794us/sample - loss: 0.3037 - accuracy: 0.8314
Epoch 379/400
1000/1000 [==============================] - 1s 708us/sample - loss: 0.2982 - accuracy: 0.8374
Epoch 380/400
1000/1000 [==============================] - 1s 863us/sample - loss: 0.2992 - accuracy: 0.8353
Epoch 381/400
1000/1000 [==============================] - 1s 760us/sample - loss: 0.3043 - accuracy: 0.8341
Epoch 382/400
1000/1000 [==============================] - 1s 778us/sample - loss: 0.3001 - accuracy: 0.8359
Epoch 383/400
1000/1000 [==============================] - 1s 968us/sample - loss: 0.3013 - accuracy: 0.8345
Epoch 384/400
1000/1000 [==============================] - 1s 879us/sample - loss: 0.3117 - accuracy: 0.8319
Epoch 385/400
1000/1000 [==============================] - 1s 1ms/sample - loss: 0.3342 - accuracy: 0.8189
Epoch 386/400
1000/1000 [============

In [42]:
#evaluate on some new patterns
n_samples = 100
X_test, y_test = generate_data(n_samples, n_numbers, largest, alphabet)
# convert input to float32 to match the ops datatype
X_test = X_test.astype(np.float32)
pred = model.predict(X_test, batch_size=n_batch, verbose=0)
# calculate error
X_test_invert = [invert(x, alphabet) for x in X_test]
expected = [float(invert(x, alphabet)) for x in y_test]
predicted = [float(invert(x, alphabet)) for x in pred]
mse = ((np.array(expected) - np.array(predicted))**2).mean(axis=0)
print("Calculation Performance in MSE : %.6f" % mse)
print("Show some specific examples")
for i in range(50):
    print('Input =%s, Expected=%s, Predicted=%s' % (X_test_invert[i], expected[i], predicted[i]))

Calculation Performance in MSE : 0.618643
Show some specific examples
Input =8.15625+2.34375, Expected=10.5, Predicted=10.4375
Input =8.40625+4.4375, Expected=12.84375, Predicted=12.625
Input =11.0625+9.125, Expected=20.1875, Predicted=20.5
Input =7.15625+2.40625, Expected=9.5625, Predicted=9.5
Input =9.4375+6.15625, Expected=15.59375, Predicted=15.5
Input =9.03125+12.375, Expected=21.40625, Predicted=20.5
Input =3.46875+6.40625, Expected=9.875, Predicted=8.96875
Input =9.28125+11.46875, Expected=20.75, Predicted=20.5625
Input =7.09375+12.5, Expected=19.59375, Predicted=19.0
Input =12.3125+5.3125, Expected=17.625, Predicted=16.59375
Input =1.4375+4.09375, Expected=5.53125, Predicted=5.125
Input =15.46875+7.28125, Expected=22.75, Predicted=21.96875
Input =9.46875+7.28125, Expected=16.75, Predicted=16.96875
Input =16.09375+5.1875, Expected=21.28125, Predicted=22.46875
Input =8.34375+9.15625, Expected=17.5, Predicted=17.46875
Input =3.15625+5.4375, Expected=8.59375, Predicted=8.84375
Inpu

In [43]:
# Task 3
# Re-do Task 2 by replacing all LSTM layers with GRU (just search tensorflow.keras.GRU on google)
# and summarize your observations in prediction performance and computation efficiency.
# Must use the same optimizer, learning rate, and number of training epochs as used in Task 2.

In [44]:
from tensorflow.keras.layers import GRU
# define GRU configuration
n_batch = 20
n_epoch = 400
# create LSTM
model = Sequential()
model.add(GRU(64, input_shape=(n_in_seq_length, n_chars)))
model.add(RepeatVector(int(n_out_seq_length)))
model.add(GRU(32, return_sequences=True))
model.add(TimeDistributed(Dense(n_chars, activation='softmax')))

optimizer = tf.keras.optimizers.Adam(lr=0.001)
model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])
print(model.summary())

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
gru (GRU)                    (None, 64)                13440     
_________________________________________________________________
repeat_vector_2 (RepeatVecto (None, 10, 64)            0         
_________________________________________________________________
gru_1 (GRU)                  (None, 10, 32)            9408      
_________________________________________________________________
time_distributed_2 (TimeDist (None, 10, 4)             132       
Total params: 22,980
Trainable params: 22,980
Non-trainable params: 0
_________________________________________________________________
None


In [45]:
model.fit(X_train, y_train, epochs=n_epoch, batch_size=n_batch, verbose=1)

Train on 1000 samples
Epoch 1/400
1000/1000 [==============================] - 4s 4ms/sample - loss: 0.8410 - accuracy: 0.5137
Epoch 2/400
1000/1000 [==============================] - 1s 911us/sample - loss: 0.6936 - accuracy: 0.5191
Epoch 3/400
1000/1000 [==============================] - 1s 1ms/sample - loss: 0.6934 - accuracy: 0.5154
Epoch 4/400
1000/1000 [==============================] - 1s 1ms/sample - loss: 0.6931 - accuracy: 0.5119
Epoch 5/400
1000/1000 [==============================] - 1s 996us/sample - loss: 0.6930 - accuracy: 0.5136
Epoch 6/400
1000/1000 [==============================] - 1s 981us/sample - loss: 0.6922 - accuracy: 0.5166
Epoch 7/400
1000/1000 [==============================] - 1s 1ms/sample - loss: 0.6925 - accuracy: 0.5165
Epoch 8/400
1000/1000 [==============================] - 1s 971us/sample - loss: 0.6927 - accuracy: 0.5150
Epoch 9/400
1000/1000 [==============================] - 1s 915us/sample - loss: 0.6923 - accuracy: 0.5109
Epoch 10/400
1000/1000 

1000/1000 [==============================] - 1s 917us/sample - loss: 0.5314 - accuracy: 0.6676
Epoch 78/400
1000/1000 [==============================] - 1s 857us/sample - loss: 0.4629 - accuracy: 0.7154
Epoch 79/400
1000/1000 [==============================] - 1s 942us/sample - loss: 0.4432 - accuracy: 0.7288
Epoch 80/400
1000/1000 [==============================] - 1s 954us/sample - loss: 0.4579 - accuracy: 0.7205
Epoch 81/400
1000/1000 [==============================] - 1s 1ms/sample - loss: 0.4354 - accuracy: 0.7369
Epoch 82/400
1000/1000 [==============================] - 1s 889us/sample - loss: 0.4487 - accuracy: 0.7205
Epoch 83/400
1000/1000 [==============================] - 1s 932us/sample - loss: 0.5503 - accuracy: 0.6735
Epoch 84/400
1000/1000 [==============================] - 1s 894us/sample - loss: 0.5222 - accuracy: 0.6703
Epoch 85/400
1000/1000 [==============================] - 1s 901us/sample - loss: 0.4452 - accuracy: 0.7324
Epoch 86/400
1000/1000 [===================

1000/1000 [==============================] - 1s 1ms/sample - loss: 0.3614 - accuracy: 0.7851
Epoch 153/400
1000/1000 [==============================] - 1s 909us/sample - loss: 0.3587 - accuracy: 0.7853
Epoch 154/400
1000/1000 [==============================] - 1s 1ms/sample - loss: 0.3607 - accuracy: 0.7872
Epoch 155/400
1000/1000 [==============================] - 1s 916us/sample - loss: 0.3604 - accuracy: 0.7810
Epoch 156/400
1000/1000 [==============================] - 1s 888us/sample - loss: 0.3693 - accuracy: 0.7792
Epoch 157/400
1000/1000 [==============================] - 1s 821us/sample - loss: 0.3709 - accuracy: 0.7751
Epoch 158/400
1000/1000 [==============================] - 1s 863us/sample - loss: 0.3641 - accuracy: 0.7793
Epoch 159/400
1000/1000 [==============================] - 1s 860us/sample - loss: 0.3581 - accuracy: 0.7844
Epoch 160/400
1000/1000 [==============================] - 1s 818us/sample - loss: 0.3576 - accuracy: 0.7841
Epoch 161/400
1000/1000 [============

1000/1000 [==============================] - 1s 821us/sample - loss: 0.3157 - accuracy: 0.8175
Epoch 228/400
1000/1000 [==============================] - 1s 820us/sample - loss: 0.3178 - accuracy: 0.8122
Epoch 229/400
1000/1000 [==============================] - 1s 836us/sample - loss: 0.3205 - accuracy: 0.8149
Epoch 230/400
1000/1000 [==============================] - 1s 824us/sample - loss: 0.3177 - accuracy: 0.8128
Epoch 231/400
1000/1000 [==============================] - 1s 857us/sample - loss: 0.3206 - accuracy: 0.8107
Epoch 232/400
1000/1000 [==============================] - 1s 1ms/sample - loss: 0.3202 - accuracy: 0.8106
Epoch 233/400
1000/1000 [==============================] - 2s 2ms/sample - loss: 0.3160 - accuracy: 0.8182
Epoch 234/400
1000/1000 [==============================] - 1s 1ms/sample - loss: 0.3348 - accuracy: 0.8061
Epoch 235/400
1000/1000 [==============================] - 2s 2ms/sample - loss: 0.3204 - accuracy: 0.8114
Epoch 236/400
1000/1000 [================

1000/1000 [==============================] - 1s 1ms/sample - loss: 0.2979 - accuracy: 0.8295
Epoch 303/400
1000/1000 [==============================] - 1s 842us/sample - loss: 0.2964 - accuracy: 0.8306
Epoch 304/400
1000/1000 [==============================] - 1s 972us/sample - loss: 0.2982 - accuracy: 0.8278
Epoch 305/400
1000/1000 [==============================] - 1s 790us/sample - loss: 0.3147 - accuracy: 0.8239
Epoch 306/400
1000/1000 [==============================] - 1s 764us/sample - loss: 0.3545 - accuracy: 0.8019
Epoch 307/400
1000/1000 [==============================] - 1s 782us/sample - loss: 0.3177 - accuracy: 0.8181
Epoch 308/400
1000/1000 [==============================] - 1s 811us/sample - loss: 0.2934 - accuracy: 0.8296
Epoch 309/400
1000/1000 [==============================] - 1s 785us/sample - loss: 0.2864 - accuracy: 0.8372
Epoch 310/400
1000/1000 [==============================] - 1s 879us/sample - loss: 0.2819 - accuracy: 0.8398
Epoch 311/400
1000/1000 [==========

1000/1000 [==============================] - 1s 728us/sample - loss: 0.2545 - accuracy: 0.8567
Epoch 378/400
1000/1000 [==============================] - 1s 756us/sample - loss: 0.2620 - accuracy: 0.8554
Epoch 379/400
1000/1000 [==============================] - 1s 764us/sample - loss: 0.2857 - accuracy: 0.8502
Epoch 380/400
1000/1000 [==============================] - 1s 776us/sample - loss: 0.2641 - accuracy: 0.8568
Epoch 381/400
1000/1000 [==============================] - 1s 758us/sample - loss: 0.2573 - accuracy: 0.8600
Epoch 382/400
1000/1000 [==============================] - 1s 755us/sample - loss: 0.2550 - accuracy: 0.8601
Epoch 383/400
1000/1000 [==============================] - 1s 739us/sample - loss: 0.2518 - accuracy: 0.8636
Epoch 384/400
1000/1000 [==============================] - 1s 740us/sample - loss: 0.2571 - accuracy: 0.8562
Epoch 385/400
1000/1000 [==============================] - 1s 744us/sample - loss: 0.2559 - accuracy: 0.8632
Epoch 386/400
1000/1000 [========

In [115]:
#evaluate on some new patterns
n_samples = 100
X_test, y_test = generate_data(n_samples, n_numbers, largest, alphabet)
# convert input to float32 to match the ops datatype
X_test = X_test.astype(np.float32)
pred = model.predict(X_test, batch_size=n_batch, verbose=0)
# calculate error
X_test_invert = [invert(x, alphabet) for x in X_test]
expected = [float(invert(x, alphabet)) for x in y_test]
predicted = [float(invert(x, alphabet)) for x in pred]
mse = ((np.array(expected) - np.array(predicted))**2).mean(axis=0)
print("Calculation Performance in MSE : %.6f" % mse)
print("Show some specific examples")
for i in range(50):
    print('Input =%s, Expected=%s, Predicted=%s' % (X_test_invert[i], expected[i], predicted[i]))

Calculation Performance in MSE : 0.310371
Show some specific examples
Input =11.21875+3.46875, Expected=14.6875, Predicted=14.5
Input =3.5+16.34375, Expected=19.84375, Predicted=19.5
Input =15.375+12.375, Expected=27.75, Predicted=27.875
Input =10.3125+16.28125, Expected=26.59375, Predicted=26.75
Input =4.4375+4.25, Expected=8.6875, Predicted=8.71875
Input =9.21875+14.4375, Expected=23.65625, Predicted=23.5
Input =16.21875+8.125, Expected=24.34375, Predicted=24.5
Input =13.03125+11.0625, Expected=24.09375, Predicted=24.40625
Input =8.21875+3.34375, Expected=11.5625, Predicted=11.46875
Input =4.03125+13.5, Expected=17.53125, Predicted=17.84375
Input =7.21875+13.09375, Expected=20.3125, Predicted=20.21875
Input =11.3125+2.4375, Expected=13.75, Predicted=13.59375
Input =4.5+9.09375, Expected=13.59375, Predicted=13.5
Input =5.46875+2.125, Expected=7.59375, Predicted=7.78125
Input =12.34375+6.40625, Expected=18.75, Predicted=18.625
Input =14.46875+6.1875, Expected=20.65625, Predicted=20.75


In [ ]:
### Comment on LSTM vs. GRU models 
# The GRU model trained faster than the LSTM.
# When it comes to testing, the GRU model performs better than the LSTM as well. 
# This can be seen from the comparison of the MSE values for each model.